In [11]:
import ddqn
import gym 
import time
from IPython.display import clear_output
import numpy as np

from utils import show_RAM_usage

# setup
# parameters
REPLAYMEMORY_SIZE = 1000000
EPSILON_DECAY = 10000
TARGETUPDATERATE = 1000
BATCHSIZE = 32
WARMUPSTEPS = 1000
TRAINSTEPS = 4
GAMMA = 0.99
RENDER = True

# agent
env = gym.make('Breakout-ram-v0')

In [29]:
env.reset()
env.step(1)
env.observation_space.shape
show_RAM_usage()

RAM usage: 1.32 GB 11.8 %


In [14]:

version = 'ddqn-0.2'

agent = ddqn.brain(env, version, REPLAYMEMORY_SIZE, TARGETUPDATERATE, BATCHSIZE, WARMUPSTEPS, TRAINSTEPS, GAMMA, RENDER, EPSILON_DECAY)
agent.main_network = agent.create_model()
agent.target_network = agent.create_model()


In [28]:
np.reshape(nextState, (1, 128))

array([[ 63,  63,  63,  63,  63,  63, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 192, 192, 192, 192, 192, 192, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 240,   0,   0,
        255,   0,   0, 240,   0,   5,   0,   0,   6,   0,  70, 182, 134,
        198,  22,  38,  54,  70,  88,   4, 146,   0,   6,   0,   0,   0,
          0,   0,   0, 241,   0, 242,   0, 242,  25, 241,   5, 242,   2,
          0, 255,   0, 226,  71,   0,   0,   0, 129,   0, 113,   0,   1,
          0, 255,   0,   0,   8,   0, 255, 255, 255, 255, 255, 255, 255,
          0,   0,   5,   0,   0, 186, 214, 117, 246, 219, 242]],
      dtype=uint8)

In [22]:
agent.main_network.predict(np.reshape(nextState, (1, 128)))

array([[  91.2608  ,  -54.904446, -244.16428 , -332.17197 ]],
      dtype=float32)

In [24]:
import random 

for i in range(10):
    print(random.randrange(0, 4))

3
3
0
2
3
2
2
3
2
2


In [15]:
#run
agent.epsilon = 0.8
EPISODES = 200
start = time.time()
for e in range(EPISODES):
    state = env.reset()
    steps = 0
    terminal = False
    episodic_reward = 0
    while not terminal:
        steps += 1
        #env.render()
        action = agent.get_action(state)
        nextState, reward, terminal, _ = env.step(action)
        agent.set_perception(state, action, reward, nextState, terminal)
        state = nextState
        episodic_reward += reward
        clear_output()
        show_RAM_usage()
        print(action, terminal)
        print('episode {} / {} step {} reward {} epsilon {}  \r'.format(e, EPISODES, steps, episodic_reward, agent.epsilon))
        if steps >= 200:
            break
    agent.logger['episodic_rewards'].append(steps)
print('end of training')
print('runtime {} minutes'.format(round(time.time()-start)/60))
env.close()
agent.save_experiment()


# setup
# parameters
REPLAYMEMORY_SIZE = 1000000
EPSILON_DECAY = 100000
TARGETUPDATERATE = 10000
BATCHSIZE = 32
WARMUPSTEPS = 10000
TRAINSTEPS = 4
GAMMA = 0.99
RENDER = False

# agent
env = gym.make('Breakout-ram-v0')
version = 'ddqn.0.1'

agent = ddqn.brain(env, version, REPLAYMEMORY_SIZE, TARGETUPDATERATE, BATCHSIZE, WARMUPSTEPS, TRAINSTEPS, GAMMA, RENDER, EPSILON_DECAY)
agent.main_network = agent.create_model()
agent.target_network = agent.create_model()

#run
EPISODES = 10000
past_30_actions = deque(maxlen=30)
start = time.time()
for e in range(EPISODES):
    state = env.reset()
    steps = 0
    terminal = False
    episodic_reward = 0
    while not terminal:
        steps += 1
        action = agent.get_action(state)
        
        # stop episode if agent does not react
        past_30_actions.append(action)
        if (len(set(past_30_actions)) == 1) & (len(past_30_actions) == 30):
            terminal = True
        
        nextState, reward, terminal, _ = env.step(action)
        agent.set_perception(state, action, reward, nextState, terminal)
        state = nextState
        episodic_reward += reward
        
        # safety net
        if get_RAM_usage() >= 98:
            agent.save_experiment()
            
        show_RAM_usage()    
        print('episode {} / {} steps {} reward {} epsilon {}  \r'.format(e, EPISODES, steps, episodic_reward, agent.epsilon))
    # end of episode    
    agent.logger['episodic_rewards'].append(episodic_reward)
print('end of training')
agent.logger['runtime'] = round(time.time()-start)/60
print('runtime {} minutes'.format(round(time.time()-start)/60))
env.close()
agent.save_experiment()

RAM usage: 1.65 GB 7.6 %
0 False
episode 5 / 200 step 43 reward 0.0 epsilon 0.7956000000000005  
start training agent


KeyboardInterrupt: 

In [10]:
from utils import get_RAM_usage

get_RAM_usage()

7.8

In [8]:
past_30_actions.append(1)
print(len(past_30_actions),len(set(past_30_actions)))

7 2
